# Task 2: 
1. Retrain the model (on the same training and validation dataset) with ‘dice_loss` as the loss function. 
2. The metrics should include at least `dice_score` and `IoU`.
3. Keep model’s input_size as 320x320.


### To Retrain require the path of the saved model. (.pth , or any checkpoint saved )
### 1. You can either train over your own model
### 2. Train over the original weights of model
<br>
download the model weight from the github its present <br>
https://github.com/xuebinqin/U-2-Net

In [10]:
pwd


'/kaggle/working'

In [11]:
!git clone https://github.com/xuebinqin/U-2-Net.git


fatal: destination path 'U-2-Net' already exists and is not an empty directory.


In [14]:
%cd U-2-Net
%pwd

/kaggle/working/U-2-Net


'/kaggle/working/U-2-Net'

In [15]:
import os
import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.optim as optim
import torchvision.transforms as standard_transforms

import numpy as np
import glob
import os
import torch.nn.functional as F

from data_loader import Rescale
from data_loader import RescaleT
from data_loader import RandomCrop
from data_loader import ToTensor
from data_loader import ToTensorLab
from data_loader import SalObjDataset

from model import U2NET
from model import U2NETP
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Change accordingly 

In [17]:
# mount drive with dataset or upload folder
# change accordingly model path and the image path
image_dir = "/kaggle/input/carmasks/dataset-20230907T065449Z-001/dataset/Image/"
label_dir = "/kaggle/input/carmasks/dataset-20230907T065449Z-001/dataset/Mask/"
model_path = "/kaggle/input/carmasks/u2net.pth"

In [32]:
#config class
class cfg:
  epoch_num = 50
  batch_size_train = 4
  batch_size_val = 1
  train_num = 0
  val_num = 0
  image_ext='.jpg'
  label_ext ='.png'
  model_name = 'u2net'

cfg.image_dir = image_dir
cfg.label_dir = label_dir
cfg.model_dir  = os.getcwd()



In [33]:
tra_img_name_list = glob.glob(cfg.image_dir + '*' + cfg.image_ext)
tra_lbl_name_list = []
for img_path in tra_img_name_list:
	img_name = img_path.split(os.sep)[-1]

	aaa = img_name.split(".")
	bbb = aaa[0:-1]
	imidx = bbb[0]
	for i in range(1,len(bbb)):
		imidx = imidx + "." + bbb[i]

	tra_lbl_name_list.append(cfg.label_dir + imidx + cfg.label_ext)

In [34]:
tra_img_train_list, tra_img_val_list, tra_lbl_train_list, tra_lbl_val_list = train_test_split(
    tra_img_name_list, tra_lbl_name_list, test_size=5, random_state=42)

In [35]:
salobj_train_dataset = SalObjDataset(
    img_name_list=tra_img_train_list,
    lbl_name_list=tra_lbl_train_list,
    transform=transforms.Compose([
        RescaleT(320),
        RandomCrop(288),
        ToTensorLab(flag=0)]))

salobj_val_dataset = SalObjDataset(
    img_name_list=tra_img_val_list,
    lbl_name_list=tra_lbl_val_list,
    transform=transforms.Compose([
        RescaleT(320),
        RandomCrop(288),
        ToTensorLab(flag=0)]))

salobj_train_dataloader = DataLoader(salobj_train_dataset, batch_size=cfg.batch_size_train, shuffle=True, num_workers=1)
salobj_val_dataloader = DataLoader(salobj_val_dataset, batch_size=cfg.batch_size_val, shuffle=False, num_workers=1)

## Loss functions 

In [36]:
bce_loss = nn.BCELoss(size_average=True)

def muti_bce_loss_fusion(d0, d1, d2, d3, d4, d5, d6, labels_v):

	loss0 = bce_loss(d0,labels_v)
	loss1 = bce_loss(d1,labels_v)
	loss2 = bce_loss(d2,labels_v)
	loss3 = bce_loss(d3,labels_v)
	loss4 = bce_loss(d4,labels_v)
	loss5 = bce_loss(d5,labels_v)
	loss6 = bce_loss(d6,labels_v)

	loss = loss0 + loss1 + loss2 + loss3 + loss4 + loss5 + loss6
	print("l0: %3f, l1: %3f, l2: %3f, l3: %3f, l4: %3f, l5: %3f, l6: %3f\n"%(loss0.data.item(),loss1.data.item(),loss2.data.item(),loss3.data.item(),loss4.data.item(),loss5.data.item(),loss6.data.item()))

	return loss0, loss

In [37]:
class DiceLoss(torch.nn.Module):
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, prediction, target):
        intersection = torch.sum(prediction * target)
        union = torch.sum(prediction) + torch.sum(target)
        dice_coeff = (2.0 * intersection + self.smooth) / (union + self.smooth)
        loss = 1.0 - dice_coeff
        return loss

In [38]:
def muti_Dice_loss_fusion(d0, d1, d2, d3, d4, d5, d6, labels_v):

	loss0 = DiceLoss(smooth=1e-5)(d0,labels_v)
	loss1 = DiceLoss(smooth=1e-5)(d1,labels_v)
	loss2 = DiceLoss(smooth=1e-5)(d2,labels_v)
	loss3 = DiceLoss(smooth=1e-5)(d3,labels_v)
	loss4 = DiceLoss(smooth=1e-5)(d4,labels_v)
	loss5 = DiceLoss(smooth=1e-5)(d5,labels_v)
	loss6 = DiceLoss(smooth=1e-5)(d6,labels_v)

	loss = loss0 + loss1 + loss2 + loss3 + loss4 + loss5 + loss6
	print("l0: %3f, l1: %3f, l2: %3f, l3: %3f, l4: %3f, l5: %3f, l6: %3f\n"%(loss0.data.item(),loss1.data.item(),loss2.data.item(),loss3.data.item(),loss4.data.item(),loss5.data.item(),loss6.data.item()))

	return loss0, loss

In [39]:
#IOU loss for the item .....
class IoULoss(torch.nn.Module):
    def __init__(self, smooth=1.0):
        super(IoULoss, self).__init__()
        self.smooth = smooth

    def forward(self, prediction, target):
        intersection = torch.sum(prediction * target)
        union = torch.sum(prediction) + torch.sum(target) - intersection
        iou = (intersection + self.smooth) / (union + self.smooth)
        loss = 1.0 - iou
        return loss

def multi_IoU_loss_fusion(d0, d1, d2, d3, d4, d5, d6, labels_v):
    loss0 = IoULoss(smooth=1e-5)(d0, labels_v)
    loss1 = IoULoss(smooth=1e-5)(d1, labels_v)
    loss2 = IoULoss(smooth=1e-5)(d2, labels_v)
    loss3 = IoULoss(smooth=1e-5)(d3, labels_v)
    loss4 = IoULoss(smooth=1e-5)(d4, labels_v)
    loss5 = IoULoss(smooth=1e-5)(d5, labels_v)
    loss6 = IoULoss(smooth=1e-5)(d6, labels_v)

    loss = loss0 + loss1 + loss2 + loss3 + loss4 + loss5 + loss6
    print("l0: %3f, l1: %3f, l2: %3f, l3: %3f, l4: %3f, l5: %3f, l6: %3f\n" % (
        loss0.data.item(), loss1.data.item(), loss2.data.item(), loss3.data.item(), loss4.data.item(), loss5.data.item(),
        loss6.data.item()))


# Retrain with the best model with least validation losss.....

In [40]:
if(cfg.model_name=='u2net'):
    net = U2NET(3, 1)
elif(cfg.model_name=='u2netp'):
    net = U2NETP(3,1)
if torch.cuda.is_available():
    net.cuda()


1. You can either train over your own model
2. Train over the original weights of model

In [41]:
## 1. Own Model ..
# checkpoint_path = '/content/drive/MyDrive/BestModel/U-2-Netu2net_best.pth'

## 2. Original weights
# checkpoint_path = '/content/drive/MyDrive/BestModel/u2net.pth'
net.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [42]:
print("---define optimizer...")
optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)

---define optimizer...


In [43]:
# ------- 5. training process --------
ite_num = 0
running_loss = 0.0
running_tar_loss = 0.0
ite_num4val = 0
save_frq = 10
best_val_loss=999999 #largeno
print("---start training...")
for epoch in range(0, cfg.epoch_num):
    net.train()

    for i, data in enumerate(salobj_train_dataloader):
        ite_num = ite_num + 1
        ite_num4val = ite_num4val + 1

        inputs, labels = data['image'], data['label']

        inputs = inputs.type(torch.FloatTensor)
        labels = labels.type(torch.FloatTensor)

        # wrap them in Variable
        if torch.cuda.is_available():
            inputs_v, labels_v = Variable(inputs.cuda(), requires_grad=False), Variable(labels.cuda(),
                                                                                        requires_grad=False)
        else:
            inputs_v, labels_v = Variable(inputs, requires_grad=False), Variable(labels, requires_grad=False)

        # y zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        d0, d1, d2, d3, d4, d5, d6 = net(inputs_v)
        loss2, loss = muti_Dice_loss_fusion(d0, d1, d2, d3, d4, d5, d6, labels_v)

        loss.backward()
        optimizer.step()

        # # print statistics
        running_loss += loss.data.item()
        running_tar_loss += loss2.data.item()

        # del temporary outputs and loss
        del d0, d1, d2, d3, d4, d5, d6, loss2, loss

        print("[epoch: %3d/%3d, batch: %5d/%5d, ite: %d] train loss: %3f, tar: %3f " % (
        epoch + 1, cfg.epoch_num, (i + 1) * cfg.batch_size_train, cfg.train_num, ite_num, running_loss / ite_num4val, running_tar_loss / ite_num4val))

        if ite_num % save_frq == 0:

            torch.save(net.state_dict(), cfg.model_dir + cfg.model_name+"_bce_itr_%d_train_%3f_tar_%3fRetrain.pth" % (ite_num, running_loss / ite_num4val, running_tar_loss / ite_num4val))
            running_loss = 0.0
            running_tar_loss = 0.0
            net.train()  # resume train
            ite_num4val = 0

        ## validation
        # Validation loop
        net.eval()  # Set the model to evaluation mode
        val_loss = 0.0

        with torch.no_grad():
            for i, data in enumerate(salobj_val_dataloader):
                inputs, labels = data['image'], data['label']

                inputs = inputs.type(torch.FloatTensor)
                labels = labels.type(torch.FloatTensor)

                if torch.cuda.is_available():
                    inputs_v, labels_v = Variable(inputs.cuda(), requires_grad=False), Variable(labels.cuda(), requires_grad=False)
                else:
                    inputs_v, labels_v = Variable(inputs, requires_grad=False), Variable(labels, requires_grad=False)

                # Forward pass
                d0, d1, d2, d3, d4, d5, d6 = net(inputs_v)
                loss2, loss = muti_Dice_loss_fusion(d0, d1, d2, d3, d4, d5, d6, labels_v)

                val_loss += loss.data.item()

                # Clean up
                del d0, d1, d2, d3, d4, d5, d6, loss2, loss

        average_val_loss = val_loss / len(salobj_val_dataloader)
        print("[epoch: %3d/%3d] Validation Loss: %3f" % (epoch + 1, cfg.epoch_num, average_val_loss))

        # Save the model checkpoint if validation loss is lower than previous best
        if average_val_loss < best_val_loss:
            best_val_loss = average_val_loss
            torch.save(net.state_dict(), cfg.model_dir + cfg.model_name + "Retrain_best.pth")

        net.train()  # S


---start training...
l0: 0.031414, l1: 0.031429, l2: 0.032272, l3: 0.032020, l4: 0.035848, l5: 0.037846, l6: 0.067728

[epoch:   1/ 50, batch:     4/    0, ite: 1] train loss: 0.268557, tar: 0.031414 
l0: 0.068745, l1: 0.069223, l2: 0.070167, l3: 0.072453, l4: 0.056569, l5: 0.072215, l6: 0.109578

l0: 0.007252, l1: 0.007254, l2: 0.007659, l3: 0.008265, l4: 0.010099, l5: 0.013139, l6: 0.021433

l0: 0.044859, l1: 0.044799, l2: 0.046144, l3: 0.046454, l4: 0.047927, l5: 0.050784, l6: 0.061271

l0: 0.011065, l1: 0.011036, l2: 0.011905, l3: 0.012704, l4: 0.016321, l5: 0.021500, l6: 0.031684

l0: 0.168640, l1: 0.168754, l2: 0.166885, l3: 0.170490, l4: 0.155342, l5: 0.142701, l6: 0.116723

[epoch:   1/ 50] Validation Loss: 0.428407
l0: 0.014854, l1: 0.014840, l2: 0.015413, l3: 0.016729, l4: 0.019884, l5: 0.027462, l6: 0.048061

[epoch:   1/ 50, batch:     8/    0, ite: 2] train loss: 0.212901, tar: 0.023134 
l0: 0.088174, l1: 0.088138, l2: 0.088885, l3: 0.089835, l4: 0.087048, l5: 0.086689, l6

In [ ]:
# Save Best model to the drive :
# import shutil
# shutil.copyfile('/content/U-2-Netu2netRetrain_best.pth', '/content/drive/MyDrive/BestModel/U-2-Netu2net_best_Retrain.pth')

# Model working Output Examin :

| Image | Output |
|-------|--------|
| ![Image](https://github.com/3112ik09/Images/blob/main/17.jpg?raw=true) | ![Output](https://github.com/3112ik09/Images/blob/main/retrainTest.png?raw=true) |

## Check in Unseen Image : 
| Input Image | Output |
|-------------|--------|
| ![Image](https://github.com/3112ik09/Images/blob/main/car1.jpg?raw=true) | ![Output](https://github.com/3112ik09/Images/blob/main/retrainUnseen.png?raw=true) |


### ADVT: 
##### This Model works well to dedect the vehicle on unseen data .

### Dis. ADVT:
##### can dedect other oubject tooo,..
